In [1]:
from datetime import datetime

In [2]:
# import secrets and tokens from config.py
from config import client_id, client_secret, refresh_token

import requests
import urllib3

# import packages for data manipulation
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

payload = {
    'client_id': client_id,
    'client_secret': client_secret,
    'refresh_token': refresh_token,
    'grant_type': "refresh_token",
    'f': 'json'
}

In [3]:
print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_dataset = requests.get(activites_url, headers=header, params=param).json()

Requesting Token...

Access Token = 98c993682cdc78cdb62aef57dde08c6fcd7b4329



In [4]:
activities = pd.json_normalize(my_dataset)

In [5]:
activities.columns 

Index(['resource_state', 'name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'sport_type', 'workout_type', 'id',
       'start_date', 'start_date_local', 'timezone', 'utc_offset',
       'location_city', 'location_state', 'location_country',
       'achievement_count', 'kudos_count', 'comment_count', 'athlete_count',
       'photo_count', 'trainer', 'commute', 'manual', 'private', 'visibility',
       'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'average_speed',
       'max_speed', 'average_cadence', 'average_watts', 'max_watts',
       'weighted_average_watts', 'kilojoules', 'device_watts', 'has_heartrate',
       'average_heartrate', 'max_heartrate', 'heartrate_opt_out',
       'display_hide_heartrate_option', 'elev_high', 'elev_low', 'upload_id',
       'upload_id_str', 'external_id', 'from_accepted_tag', 'pr_count',
       'total_photo_count', 'has_kudoed', 'suffer_score', 'athlete.id',
       'athlete.resource_state', 'map.id', 'map.sum

In [6]:
#Break date into start time and date
activities['start_date_local'] = pd.to_datetime(activities['start_date_local'])
activities['start_time'] = activities['start_date_local'].dt.time
activities['start_date_local'] = activities['start_date_local'].dt.date
activities.head(5)

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,...,total_photo_count,has_kudoed,suffer_score,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state,average_temp,start_time
0,2,Morning Run,8744.9,2998,3030,19.0,Run,Run,NaN,9659475315,...,0,False,83.0,43537797,1,a9659475315,ahgjF|irdVG~@s@fGK\OZU`Ac@jDKpAIj@G`@EFSDi@Uw@...,2,NaN,05:58:03
1,2,Morning Run,6497.6,2385,2385,8.0,Run,Run,NaN,9645925288,...,0,False,33.0,43537797,1,a9645925288,sqmjFlhjdVR@XH^PXFn@Xn@Nn@EH@tA`@l@MJGZ]JYv@{E...,2,NaN,06:39:39
2,2,Morning Run,16107.1,5309,5322,39.0,Run,Run,NaN,9640679029,...,0,False,223.0,43537797,1,a9640679029,o{wjF~zccVoAEaB_@_Bu@_AaAq@_@cCwE}CiHKMg@Me@Fe...,2,NaN,07:13:58
3,2,Morning Stand Up Paddling,5224.9,5882,6228,19.7,StandUpPaddling,StandUpPaddling,NaN,9635691824,...,0,False,11.0,43537797,1,a9635691824,kpvhFrhjiVVJDFRLN@NGF?PQTKPELG@E?BLDCBG@@BPUJC...,2,NaN,10:10:31
4,2,Afternoon Weight Training,0.0,2903,2903,0.0,WeightTraining,WeightTraining,NaN,9628698824,...,0,False,10.0,43537797,1,a9628698824,,2,NaN,13:08:44


In [7]:
# Make a copy of activities DataFrame for testing.
activities_copy = activities.copy()

In [8]:
activities_copy.drop(
    [
        'map.summary_polyline', 
        'resource_state',
        'external_id', 
        'upload_id', 
        'location_city', 
        'location_state', 
        'has_kudoed', 
        'start_date', 
        'athlete.resource_state', 
        'utc_offset', 
        'map.resource_state', 
        'athlete.id', 
        'visibility', 
        'heartrate_opt_out', 
        'upload_id_str', 
        'from_accepted_tag', 
        'map.id', 
        'manual', 
        'private', 
        'flagged', 
    ], 
    axis=1, 
    inplace=True
)

In [9]:
# filter down to Ride, Run, and Swim activities
activities_copy = activities_copy.query("type == 'Run'")
print(len(activities_copy))

114


In [10]:
# convert data types

# what does this do?
# activities_copy.loc[:, 'start_date'] = pd.to_datetime(activities_copy['start_date']).dt.tz_localize(None)
# what does this do?
activities_copy.loc[:, 'start_date_local'] = pd.to_datetime(activities_copy['start_date_local']).dt.tz_localize(None)
# convert values
activities_copy.loc[:, 'distance'] /= 1609.344 # convert from meters to miles
activities_copy.loc[:, 'average_speed'] *= 2.23693629 # convert from meters/second to miles/hour
activities_copy.loc[:, 'max_speed'] *= 2.23693629 # convert from meters/second to miles/hour
# set index
#activities_copy.set_index('start_date_local', inplace=True)

C:\Users\jimmy\AppData\Local\Temp\ipykernel_42208\4159416693.py:6: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  activities_copy.loc[:, 'start_date_local'] = pd.to_datetime(activities_copy['start_date_local']).dt.tz_localize(None)


In [11]:
activities_copy.head()

,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,start_date_local,...,average_heartrate,max_heartrate,display_hide_heartrate_option,elev_high,elev_low,pr_count,total_photo_count,suffer_score,average_temp,start_time
0,Morning Run,5.433829,2998,3030,19.0,Run,Run,NaN,9659475315,2023-08-16,...,156.0,180.0,True,11.2,2.8,0,0,83.0,NaN,05:58:03
1,Morning Run,4.037421,2385,2385,8.0,Run,Run,NaN,9645925288,2023-08-14,...,142.5,157.0,True,12.0,7.4,4,0,33.0,NaN,06:39:39
2,Morning Run,10.008488,5309,5322,39.0,Run,Run,NaN,9640679029,2023-08-13,...,166.1,176.0,True,32.6,16.8,1,0,223.0,NaN,07:13:58
5,Morning Run,4.011883,2354,2371,6.0,Run,Run,NaN,9620123035,2023-08-10,...,135.6,145.0,True,11.6,5.8,4,0,16.0,NaN,06:32:09
6,Evening Run,4.116771,1994,2029,7.0,Run,Run,NaN,9617068068,2023-08-09,...,165.4,184.0,True,13.6,5.8,2,0,84.0,NaN,18:12:25


In [12]:
# https://stackoverflow.com/questions/25146121/extracting-just-month-and-year-separately-from-pandas-datetime-column
# create a column that extracts month and year from the activity
# df['yyyy-mm'] = pd.to_datetime(df['ArrivalDate']).dt.strftime('%Y-%m')
activities_copy['Month_Year'] = pd.to_datetime(activities_copy['start_date_local']).dt.strftime('%Y-%m')

In [13]:
# https://stackoverflow.com/questions/2600775/how-to-get-week-number-in-python
# make a Week_Of_Year column
activities_copy['Week_Of_Year'] = pd.to_datetime(activities_copy['start_date_local']).dt.strftime('%U')

In [14]:
# sort by start_date_local asc

In [15]:
# get weekly mileage and total weekly moving time
df_miles_per_week = pd.DataFrame(activities_copy.groupby(['Week_Of_Year'])['distance', 'moving_time'].sum().reset_index())

C:\Users\jimmy\AppData\Local\Temp\ipykernel_42208\229282904.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_miles_per_week = pd.DataFrame(activities_copy.groupby(['Week_Of_Year'])['distance', 'moving_time'].sum().reset_index())


In [16]:
# make a average mph column
df_miles_per_week['Average Moving Speed (mph)'] = df_miles_per_week['distance']/(df_miles_per_week['moving_time']/(60*60))

In [17]:
# option: make a column that gives the first day of the week
# lookup Week_Of_Year???

In [18]:
# rename columns
df_miles_per_week.rename(columns={"distance": "Weekly Mileage", "moving_time": "Total Moving Time (seconds)"}).tail()

,Week_Of_Year,Weekly Mileage,Total Moving Time (seconds),Average Moving Speed (mph)
26,29,10.534914,5892,6.436811
27,30,9.913045,5226,6.828734
28,31,19.129347,10300,6.685985
29,32,14.770677,8366,6.356017
30,33,19.479738,10692,6.558834


In [19]:
# Make function that calculates weekly mileage targets
def weekly_mileage_target(easy_runs_minutes, long_run_mileage):

    easy_miles = easy_runs_minutes/10
    total_weekly_mileage = easy_miles + long_run_mileage
    return total_weekly_mileage

In [20]:
# convert the Week_Of_Year column to numeric type so we can filter on it
df_miles_per_week["Week_Of_Year"] = pd.to_numeric(df_miles_per_week["Week_Of_Year"])


In [21]:
# sort down to marathon training start
df_marathon = df_miles_per_week.query("Week_Of_Year >= 31")
df_marathon.head()
# calculate difference to targets

,Week_Of_Year,distance,moving_time,Average Moving Speed (mph)
28,31,19.129347,10300,6.685985
29,32,14.770677,8366,6.356017
30,33,19.479738,10692,6.558834


In [22]:
# Make function that calculates weekly mileage targets
def weekly_mileage_target(easy_runs_minutes, long_run_mileage):

    easy_miles = easy_runs_minutes/10
    total_weekly_mileage = easy_miles + long_run_mileage
    return total_weekly_mileage

In [23]:
week_1_goal_mileage = weekly_mileage_target(190, 8)
week_2_goal_mileage = weekly_mileage_target(190, 10)
week_3_goal_mileage = weekly_mileage_target(200, 10)
print(week_1_goal_mileage)

27.0


In [24]:
df_goal = pd.DataFrame([week_1_goal_mileage,week_2_goal_mileage, week_3_goal_mileage], columns= ['Weekly_Mileage_Goal'])

In [25]:
df_goal['Week_Of_Year']= [31,32,33]

In [26]:
df_goal.head()

,Weekly_Mileage_Goal,Week_Of_Year
0,27.0,31
1,29.0,32
2,30.0,33


In [27]:
# merge Weekly_Mileage_Goal onto the df_marathon
df_combined = df_marathon.merge(df_goal, how = 'left', left_on='Week_Of_Year', right_on='Week_Of_Year')

In [28]:
df_combined.head()

,Week_Of_Year,distance,moving_time,Average Moving Speed (mph),Weekly_Mileage_Goal
0,31,19.129347,10300,6.685985,27.0
1,32,14.770677,8366,6.356017,29.0
2,33,19.479738,10692,6.558834,30.0


In [29]:
# plot weekly mileage
import plotly.express as px

fig0 = px.bar(
    df_marathon, x = "Week_Of_Year", y = "distance",
    labels = dict(Week_Of_Year ="Training Week", distance ="Distance (miles) "),
    #hover_data=["start_date_local"],
    title = "Weekly Mileage",
    width = 1000
)
# Hover over should be the day, not the first of the month
# fig0.add_hline(y = 20*8)
fig0.update_traces(marker_line_width = 2.5)
fig0.update_yaxes(range = [0, 50])
#fig0.update_layout(bargap = 0.8)
fig0.show() 

In [42]:
def combine_words(word_one, word_two):
    combined_word = f"{word_one} {word_two}"
    return combined_word

In [43]:
dog_description = combine_words("purple","poodle")
print(dog_description)

purple poodle


In [45]:
new_dog_description = combine_words(word_one = "blue", word_two = "heeler")
print(new_dog_description)

blue heeler


In [47]:
import random

random.seed(10)
print(random.random())

random.seed(10)
print(random.random())

random.seed(5)
print(random.random())

0.5714025946899135
0.5714025946899135
0.6229016948897019


In [48]:
def random_rgb():
    """Returns a list of random R, G, B color values.
    
    >>> random.seed(1)
    >>> random_rgb()
    [106, 184, 0]
    """
    r = random.randint(0, 255)
    g = random.randint(0, 255)
    b = random.randint(0, 255)
    return [r, g, b]

In [67]:
random_rgb()

[60, 253, 230]

In [68]:
random.seed(1)
random_rgb()

[68, 32, 130]

In [64]:
random_rgb()

[228, 136, 117]

In [59]:
random.seed(1)
random_rgb()

[68, 32, 130]